In [1]:
# Computes stochastic closure certificate using SOS for stochastic systems in verifying LTL specifications
# Software Program Modified from [8]

# include important libraries
using JuMP
using MosekTools
using DynamicPolynomials
using MultivariatePolynomials
using LinearAlgebra
using TSSOS # important for SOS, see https://github.com/wangjie212/TSSOS
using Distributions # for the noise

In [2]:
@polyvar x y x0 # global vars used in monomials
vars = [x, y]
sos_tol = 1 # the maximum degree of unknown SOS lagrangian polynomials = deg + sos_tol 
error = 2   # precision digit places
gamma = 0.04
lamda = 140
# p-sat = 99.971%
vari, mean = 0.1, 0 # variance, mean of the Guassain distribution

tau1, tau2, delta = .03, .05, 11 # S-procedure constants in condition 3
bd = 100

# build some symbolic transition variable
f_syml2 = x^2 + 1 
f_syml3 = x^2 - 1 

# noise generator
function noise()
    return rand(Normal(mean, vari))
end

# creates the parametrized SCC
function add_paramC_poly!(model, var1, deg, r, l)
    basis = monomials(var1, 0:deg) # basis in x and y
    coeffs = @variable(model, [1:length(basis)], base_name="c_$(r)_$(l)")
    C_rl = sum(coeffs[i] * basis[i] for i in 1:length(basis))
    return C_rl, coeffs, basis
end

# creates the parametrized ranking functions
function add_paramV_poly!(model, var2, deg, r, l)
    basis = monomials(var2, 0:deg)
    coeffs = @variable(model, [1:length(basis)], base_name="v_$(r)_$(l)")
    V_rl = sum(coeffs[i] * basis[i] for i in 1:length(basis))
    return V_rl, coeffs, basis
end

# we let a define x>=0 and else b
# and f2, f3 for f_syml2 and f_syml3, respectively

# the considered program P
S = Dict(
    :"l0" => [(Set(["a"]), :"l1")],
    :"l1" => [(Set(["a"]), :"l2"), (Set(["a"]), :"l3"), (Set(["b"]), :"l4")],
    :"l2" => [(Set(["f2"]), :"l1")],
    :"l3" => [(Set(["f3"]), :"l1")],
    :"l4" => [(Set([]), :"l4")]
)

Q0, QF = Set(["l0"]), Set(["l2"]) # Initial and X_Fv set of P

# considering domain X = R 
g_a1, g_a2 = [x-0, bd-x], [x-0, bd-x, y-0, bd-y] # poly describing x>=0
g_b1, g_b2 = [x+bd, 0-x], [x+bd, 0-x, y+bd, 0-y] # poly describing x<0
g_a3, g_b3 = [x0-0, 0-x0, x-0, bd-x, y-0, bd-y], [x0-0, 0-x0, x+bd, 0-x, y+bd, 0-y] 

# for the first condition (x, y)
g1 = Dict(
    :"a" => g_a1, :"f2" => g_a1, :"f3" => g_a1, :"b" => g_b1
)
# for the 2nd condition
g2 = Dict(
    :"a" => g_a2, :"f2" => g_a2, :"f3" => g_a2, :"b" => g_b2
)     
# g(x0,x,y)
# for the 3rd condition
g3 = g_a3  

# polynomial stochastic closure certificate of degree deg and NBA S
function scc(deg)
    # synthesize SCC by using the standard formulation for program
    # deg: degree of scc template
    model = Model(optimizer_with_attributes(Mosek.Optimizer))
    set_optimizer_attribute(model, MOI.Silent(), true)

    C_dict = Dict()  # key => (symbolic_poly, numeric_poly)
    
    for l in keys(S)
        for (lab_set, ln) in S[l]
            for lab in lab_set
                C, Cc, Cb = add_paramC_poly!(model, vars, deg, l, ln) # CC, CC coefficients, and CC basis functions
                if l=="l2" && ln=="l1"
                    # these modification takes care of the expectation
                    EC = C(vars=>[x,f_syml2 + noise()]) 
                    # see TSSOS https://github.com/wangjie212/TSSOS
                    key = (l, ln)  
                    C_dict[key] = (C, Cc, Cb)  # store symbolic, coefficients, basis

                    # 1st condition
                    add_psatz!(model, -EC + gamma, [x], g1[lab], [], div(deg+sos_tol,2), QUIET=true, CS=false, TS=false, GroebnerBasis=true) 
                    for r in keys(S)
                        C1, Cc1, Cb1 = add_paramC_poly!(model, vars, deg, r, ln)
                        EC1 = C1(vars=>[x,y^2 + 1 + noise()]) 
                        C2, Cc2, Cb2 = add_paramC_poly!(model, vars, deg, r, l)
                        key1, key2 = (r, ln), (r, l)  
                        C_dict[key1], C_dict[key2] = (C1, Cc1, Cb1), (C2, Cc2, Cb2) 
                        
                        # 2nd condition
                        add_psatz!(model, -EC1 + C2, [x], g2[lab], [], div(deg+sos_tol,2), QUIET=true, CS=false, TS=false, GroebnerBasis=true) 
                    end
                elseif l=="l3" && ln=="l1"
                    # these modification takes care of the expectation
                    EC = C(vars=>[x,f_syml3 + noise()]) 
                    # see TSSOS https://github.com/wangjie212/TSSOS
                    key = (l, ln)  
                    C_dict[key] = (C, Cc, Cb)  # store symbolic, coefficients, basis

                    # 1st condition
                    add_psatz!(model, -EC + gamma, [x], g1[lab], [], div(deg+sos_tol,2), QUIET=true, CS=false, TS=false, GroebnerBasis=true) 
                    for r in keys(S)
                        C1, Cc1, Cb1 = add_paramC_poly!(model, vars, deg, r, ln)
                        EC1 = C1(vars=>[x, y^2 - 1 + noise()]) 
                        C2, Cc2, Cb2 = add_paramC_poly!(model, vars, deg, r, l)
                        key1, key2 = (r, ln), (r, l)  
                        C_dict[key1], C_dict[key2] = (C1, Cc1, Cb1), (C2, Cc2, Cb2) 
                        
                        # 2nd condition
                        add_psatz!(model, -EC1 + C2, [x,y], g2[lab], [], div(deg+sos_tol,2), QUIET=true, CS=false, TS=false, GroebnerBasis=true) 
                    end
                end
            end
        end
    end
    for l0 in Q0
        for lf1 in QF
            V1, Vc1, Vb1 = add_paramV_poly!(model, [x0, x, y], deg, l0, lf1)
            key3a = (l0, lf1, "V")  
            C_dict[key3a] = (V1, Vc1, Vb1)
            for lf2 in QF
                V2, Vc2, Vb2 = add_paramV_poly!(model, [x0, x, y], deg, l0, lf2)
                C3, Cc3, Cb3 = add_paramC_poly!(model, [x0, x, y], deg, l0, lf2)
                C4, Cc4, Cb4 = add_paramC_poly!(model, [x0, x, y], deg, lf2, lf1)
                
                key3b = (l0, lf2, "V")  
                C_dict[key3b] = (V2, Vc2, Vb2)
                key4 = (l0, lf2)  
                C_dict[key4] = (C3, Cc3, Cb3)
                key5 = (lf1, lf2)  
                C_dict[key5] = (C4, Cc4, Cb4)
                # 3rd condition
                add_psatz!(model, -V1 + V2 - delta + tau1 * (lamda - C3) + tau2 * (lamda - C4), [x0, x, y], g3, [], div(deg+sos_tol,2), QUIET=true, CS=false, TS=false, GroebnerBasis=true) 
            end
        end
    end
        
    optimize!(model) # solve for coefficients
    status = termination_status(model)
    C_eval_dict = Dict() # Get numerical values of coefficients and plug into polynomials
    for (key, (C, Cc, Cb)) in C_dict
        coeff_vals = round.(value.(Cc); digits=error)  # Round each coefficient to 2 decimal places
        C_numeric = sum(coeff_vals[i] * Cb[i] for i in eachindex(Cb))
        C_eval_dict[key] = (C, C_numeric)
    end
    
    # status might be optimal but if all Bc approx 10^{-error}, it's essentially 0 so OPTIMAL != CC.
    return status, C_eval_dict
end
        

# Simulation
file = open("./systems/prog_ex_SCC_SOS.txt", "w")

deg = 1 # desired degree of SCC
stats = @timed (status, CC_data) = scc(deg)

write(file, "poly deg: "*string(deg)*"\n")
write(file, "status: "*string(status)*"\n")
write(file, "Number of SCC polynomials: "*string(length(CC_data))*"\n")
write(file, "time: "*string(stats.time)*"\n\n")

# Write the dictionary line-by-line
for (k, v) in CC_data
    write(file, "Key: $(k)\n")
    write(file, "Polynomial: $(v[1])\n")
    write(file, "Coefficients: $(v[2])\n\n")
end

close(file)

println("Finished")

Finished


#### disjunctive benefit demonstration

In [3]:
@polyvar x y x0 # global vars used in monomials
vars = [x, y]
sos_tol = 1 # the maximum degree of unknown SOS lagrangian polynomials = deg + sos_tol 
error = 2   # precision digit places
gamma = 0.015
lamda = 290
# p-sat = 99.994%
vari, mean = 0.1, 0 # variance, mean of Guassian distribution

tau1, tau2, delta = .015, .03, 16 # S-procedure constants in condition 3
bd = 100

# build some symbolic transition variable
f_syml2 = x^2 + 1 
f_syml3 = x^2 - 1 

# noise generator
function noise()
    return rand(Normal(mean, vari))
end

# creates the parametrized SCC
function add_paramC_poly!(model, var1, deg, r, l)
    basis = monomials(var1, 0:deg) # basis in x and y
    coeffs = @variable(model, [1:length(basis)], base_name="c_$(r)_$(l)")
    C_rl = sum(coeffs[i] * basis[i] for i in 1:length(basis))
    return C_rl, coeffs, basis
end

# creates the parametrized ranking functions
function add_paramV_poly!(model, var2, deg, r, l)
    basis = monomials(var2, 0:deg)
    coeffs = @variable(model, [1:length(basis)], base_name="v_$(r)_$(l)")
    V_rl = sum(coeffs[i] * basis[i] for i in 1:length(basis))
    return V_rl, coeffs, basis
end

# we let a define x>=0 and else b
# and f2, f3 for f_syml2 and f_syml3, respectively

# the considered program P
S = Dict(
    :"l0" => [(Set(["a"]), :"l1")],
    :"l1" => [(Set(["a"]), :"l2"), (Set(["a"]), :"l3"), (Set(["b"]), :"l4")],
    :"l2" => [(Set(["f2"]), :"l1")],
    :"l3" => [(Set(["f3"]), :"l1")],
    :"l4" => [(Set([]), :"l4")]
)

Q0, QF = Set(["l0"]), Set(["l2"]) # Initial and X_Fv set of P

# considering domain X = R 
g_a1, g_a2 = [x-0, bd-x], [x-0, bd-x, y-0, bd-y] # poly describing x>=0
g_b1, g_b2 = [x+bd, 0-x], [x+bd, 0-x, y+bd, 0-y] # poly describing x<0
g_a3, g_b3 = [x0-0, 0-x0, x-0, bd-x, y-0, bd-y], [x0-0, 0-x0, x+bd, 0-x, y+bd, 0-y] 

# for the first condition (x, y)
g1 = Dict(
    :"a" => g_a1, :"f2" => g_a1, :"f3" => g_a1, :"b" => g_b1
)
# for the 2nd condition
g2 = Dict(
    :"a" => g_a2, :"f2" => g_a2, :"f3" => g_a2, :"b" => g_b2
)     
# g(x0,x,y)
# for the 3rd condition X_FV= [0,50]x{l2} union [50,100]x{l2}
bd1 = 50
g31, g32 = [x0-0, 0-x0, x-0, bd1-x, y-0, bd1-y], [x0-0, 0-x0, x-bd1, bd-x, y-bd1, bd-y]  

# polynomial stochastic closure certificate of degree deg and NBA S
function scc(deg)
    # synthesize SCC by using the standard formulation for program
    # deg: degree of scc template
    model = Model(optimizer_with_attributes(Mosek.Optimizer))
    set_optimizer_attribute(model, MOI.Silent(), true)

    C_dict = Dict()  # key => (symbolic_poly, numeric_poly)
    
    for l in keys(S)
        for (lab_set, ln) in S[l]
            for lab in lab_set
                C, Cc, Cb = add_paramC_poly!(model, vars, deg, l, ln) # CC, CC coefficients, and CC basis functions
                if l=="l2" && ln=="l1"
                    # these modification takes care of the expectation
                    EC = C(vars=>[x,f_syml2 + noise()]) 
                    # see TSSOS https://github.com/wangjie212/TSSOS
                    key = (l, ln)  
                    C_dict[key] = (C, Cc, Cb)  # store symbolic, coefficients, basis

                    # 1st condition
                    add_psatz!(model, -EC + gamma, [x], g1[lab], [], div(deg+sos_tol,2), QUIET=true, CS=false, TS=false, GroebnerBasis=true) 
                    for r in keys(S)
                        C1, Cc1, Cb1 = add_paramC_poly!(model, vars, deg, r, ln)
                        EC1 = C1(vars=>[x,y^2 + 1 + noise()]) 
                        C2, Cc2, Cb2 = add_paramC_poly!(model, vars, deg, r, l)
                        key1, key2 = (r, ln), (r, l)  
                        C_dict[key1], C_dict[key2] = (C1, Cc1, Cb1), (C2, Cc2, Cb2) 
                        
                        # 2nd condition
                        add_psatz!(model, -EC1 + C2, [x], g2[lab], [], div(deg+sos_tol,2), QUIET=true, CS=false, TS=false, GroebnerBasis=true) 
                    end
                elseif l=="l3" && ln=="l1"
                    # these modification takes care of the expectation
                    EC = C(vars=>[x,f_syml3 + noise()]) 
                    # see TSSOS https://github.com/wangjie212/TSSOS
                    key = (l, ln)  
                    C_dict[key] = (C, Cc, Cb)  # store symbolic, coefficients, basis

                    # 1st condition
                    add_psatz!(model, -EC + gamma, [x], g1[lab], [], div(deg+sos_tol,2), QUIET=true, CS=false, TS=false, GroebnerBasis=true) 
                    for r in keys(S)
                        C1, Cc1, Cb1 = add_paramC_poly!(model, vars, deg, r, ln)
                        EC1 = C1(vars=>[x, y^2 - 1 + noise()]) 
                        C2, Cc2, Cb2 = add_paramC_poly!(model, vars, deg, r, l)
                        key1, key2 = (r, ln), (r, l)  
                        C_dict[key1], C_dict[key2] = (C1, Cc1, Cb1), (C2, Cc2, Cb2) 
                        
                        # 2nd condition
                        add_psatz!(model, -EC1 + C2, [x,y], g2[lab], [], div(deg+sos_tol,2), QUIET=true, CS=false, TS=false, GroebnerBasis=true) 
                    end
                end
            end
        end
    end
    # third condition over the partition
    for l0 in Q0
        for lf1 in QF 
            V1, Vc1, Vb1 = add_paramV_poly!(model, [x0, x, y], deg, l0, lf1)
            key3a = (l0, lf1, "V1a")  
            C_dict[key3a] = (V1, Vc1, Vb1)
            for lf2 in QF
                V2, Vc2, Vb2 = add_paramV_poly!(model, [x0, x, y], deg, l0, lf2)
                C3, Cc3, Cb3 = add_paramC_poly!(model, [x0, x, y], deg, l0, lf2)
                C4, Cc4, Cb4 = add_paramC_poly!(model, [x0, x, y], deg, lf2, lf1)
                
                key3b = (l0, lf2, "V1b")  
                C_dict[key3b] = (V2, Vc2, Vb2)
                key4 = (l0, lf2)  
                C_dict[key4] = (C3, Cc3, Cb3)
                key5 = (lf1, lf2)  
                C_dict[key5] = (C4, Cc4, Cb4)
                # 3rd condition
                add_psatz!(model, -V1 + V2 - delta + tau1 * (lamda - C3) + tau2 * (lamda - C4), [x0, x, y], g31, [], div(deg+sos_tol,2), QUIET=true, CS=false, TS=false, GroebnerBasis=true) 
            end
        end
    end
    for l0 in Q0
        for lf1 in QF 
            V1, Vc1, Vb1 = add_paramV_poly!(model, [x0, x, y], deg, l0, lf1)
            key3a2 = (l0, lf1, "V2a")  
            C_dict[key3a2] = (V1, Vc1, Vb1)
            for lf2 in QF
                V2, Vc2, Vb2 = add_paramV_poly!(model, [x0, x, y], deg, l0, lf2)
                C3, Cc3, Cb3 = add_paramC_poly!(model, [x0, x, y], deg, l0, lf2)
                C4, Cc4, Cb4 = add_paramC_poly!(model, [x0, x, y], deg, lf2, lf1)
                
                key3b2 = (l0, lf2, "V2b")  
                C_dict[key3b2] = (V2, Vc2, Vb2)
                key4 = (l0, lf2)  
                C_dict[key4] = (C3, Cc3, Cb3)
                key5 = (lf1, lf2)  
                C_dict[key5] = (C4, Cc4, Cb4)
                # 3rd condition
                add_psatz!(model, -V1 + V2 - delta + tau1 * (lamda - C3) + tau2 * (lamda - C4), [x0, x, y], g32, [], div(deg+sos_tol,2), QUIET=true, CS=false, TS=false, GroebnerBasis=true) 
            end
        end
    end
        
    optimize!(model) # solve for coefficients
    status = termination_status(model)
    C_eval_dict = Dict() # Get numerical values of coefficients and plug into polynomials
    for (key, (C, Cc, Cb)) in C_dict
        coeff_vals = round.(value.(Cc); digits=error)  # Round each coefficient to 2 decimal places
        C_numeric = sum(coeff_vals[i] * Cb[i] for i in eachindex(Cb))
        C_eval_dict[key] = (C, C_numeric)
    end
    
    # status might be optimal but if all Bc approx 10^{-error}, it's essentially 0 so OPTIMAL != CC.
    return status, C_eval_dict
end
        

# Simulation
file = open("./systems/prog_ex_disjunctive_SCC_SOS.txt", "w")

deg = 1 # desired degree of SCC
stats = @timed (status, CC_data) = scc(deg)

write(file, "poly deg: "*string(deg)*"\n")
write(file, "status: "*string(status)*"\n")
write(file, "Number of SCC polynomials: "*string(length(CC_data))*"\n")
write(file, "time: "*string(stats.time)*"\n\n")

# Write the dictionary line-by-line
for (k, v) in CC_data
    write(file, "Key: $(k)\n")
    write(file, "Polynomial: $(v[1])\n")
    write(file, "Coefficients: $(v[2])\n\n")
end

close(file)

println("Finished")

Finished
